In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
#import imageio
import sys 
%matplotlib notebook

# Goal
Here we want to demonstrate how our ASD-AFM aproach could be applied for experimental AFM data processing. Automated Structure Discovery means:
- identification of structure and chemical content for unknown molecule
- identification of orientations for known molecule. 

First task can be solved for more or less 2D molecules. Nevertheless it starts to feel troubles for 3D molecules. Why? It is partially connected to the nature of AFM method. Terminated by probe particle tip doesn't penetrate inside molecule. In case of bulky molecules AFM data contains only information only about some top layers of atoms. Therefore unknown molecule full chemical content identification could be problematic in some cases.

Second task is easier for any molecules with known chemical content. As a benchmark for our first iteration of the method, we apply our pretrained CNN model to resolve several distinct adsorption configurations of 1S-camphor on Cu(111) based on low-temperature AFM measurements.

The 3D experimental image stack is fed into the CNN model and a 2D image descriptor (van  der  Waals  Spheres) is predicted based on this data. This descriptor is then compared via cross-correlation to a set of descriptors calculated directly from atomic coordinates taken from a set of uniformly distributed 500 rotations of 1S-Camphor molecule in vacuum. The best fit gives us a prediction of the molecular configuration corresponding to the original vdW-Spheres descriptor from experimental data.

Experimental AFM data, simulated AFM data for 500 uniformly distributed rotations of 1S-Camphor in vacuum and also predicted from CNN vdW-Spheres image descriptors we take as an input here. Download these from here: https://www.dropbox.com/s/jq68p0u1b9hx1qu/matching_data.tar.gz?dl=0. Extract the archive to repo directory. It will create:
- simulated_configs_data.npz - simulated AFM data and predicted vdW-Spheres descriptor for 500 1S-Camphor's configurations
- simulated_configs_geom - folder with geometries for simulated configurations
- experimental_configs_data - experimental AFM data and predicted vdW-Spheres descriptors

# Experimental 1S-Camphor AFM data

Let's take a look on experimental data

In [ ]:
# plot input AFM data from preloaded to folders .npz files
def plot_experiment_AFM(list_of_exp):
    
    cols = 10
    rows = len(list_of_exp)
    fig=plt.figure(figsize=(1.8*cols, 1.8*rows))
    ax = []
    for i,experiment in enumerate(list_of_exp):
        filename_exp = 'experimental_configs_data/'+str(experiment)+'orient_exp.npz' 
        data= np.load(filename_exp)
        X_exp=data['X']
        print '#'+str(experiment)+' 1S-Champhor experiment '+ 'X.shape:', X_exp.shape
     
    
        for j in range(10):
            ax.append(fig.add_subplot(rows,cols,i*cols+j+1))
            xj = X_exp[0,:,:,j]
            vmax = xj.max()
            vmin = xj.min() 
            plt.imshow(xj,  cmap='afmhot', origin="lower",vmin=vmin-0.1*(vmax-vmin),vmax=vmax+0.1*(vmax-vmin))
            plt.xticks([])
            plt.yticks([])
            if j == 0:
                ax[-1].set_ylabel('AFM experiment '+ str(experiment))

    plt.show()
        
experiments = [1,3,4,6,7]

plot_experiment_AFM(experiments)

Even highly trained experts were not able to decipher the molecular structure from these images. 

In [ ]:
def plot_experiment_preds(list_of_exp):
    cols = len(list_of_exp)
    rows = 1
    fig=plt.figure(figsize=(2*cols, 2*rows))
    ax = []
    for i,experiment in enumerate(list_of_exp):
        filename_exp = 'experimental_configs_data/'+str(experiment)+'orient_exp.npz' 
        data= np.load(filename_exp)
        Y_exp=data['Y']
        ax.append(fig.add_subplot(rows,cols,i+1))
        plt.imshow(Y_exp[1][0], origin="lower")            
        ax[-1].set_ylabel('vdW-Spheres')
        ax[-1].set_xlabel('AFM experiment '+ str(experiment))
        plt.xticks([])
        plt.yticks([])
    plt.show()
experiments = [1,3,4,6,7]

plot_experiment_preds(experiments)

# Matching simulated configurations to experimental configurations
The best fit gives us a prediction of the molecular configuration corresponding to the original descriptor from experimental data

In [ ]:
from utils_matching import *
orientations   = [1,3, 4 ,6 ,7]
filename_model = './simulated_configs_data.npz'

for i in range(np.size(orientations)):
    orient = orientations[i]
    filename_exp = 'experimental_configs_data/'+str(orient)+'orient_exp.npz'
    # check if correlation values have been already calculated
    if os.path.isfile('experimental_configs_data/'+str(orient)+'orient_exp_sim_cor_values.csv'):
        print ('Experiment '+str(orient)+ ': correlations file exist')
    else:
        create_df_model_coef_correls(filename_model,filename_exp,orient) 
    

Let's plot for each 1S-Camphor AFM experiment few simulated configurations with best correlation coefficients:

In [ ]:
orientations    = [1,3,4,6,7]
num_best = 3 # how many best simulated configurations to plot
filename_model  = './simulated_configs_data.npz' 
dir_exp  = 'experimental_configs_data/'  # here we load csv and X Y experiment orientations data
dir_sim_geom = './simulated_configs_geom/'
plot_best_match_configs_one(filename_model,dir_sim_geom,dir_exp,orientations[0],num_best)


In [ ]:
plot_best_match_configs_one(filename_model,dir_sim_geom,dir_exp,orientations[2],num_best)


Let's plot all experiments matched in one big image:

In [ ]:
orientations    = [1,3,4,6,7]
num_best = 3 # how many best simulated configurations to plot
filename_model  = './simulated_configs_data.npz' 
dir_exp  = 'experimental_configs_data/'  # here we load csv and X Y experiment orientations data
dir_sim_geom = './simulated_configs_geom/'
plot_best_match_configs_all(filename_model,dir_sim_geom,dir_exp,orientations)
